In [0]:
dbutils.widgets.text("input_load_date", "YYYY-MM-DD", "Input Load Date")

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv

In [0]:
load_dotenv()

input_load_date = dbutils.widgets.get("input_load_date")

catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

In [0]:
silver_SP500_stock_prices_df = spark.sql(
    f"""
    WITH silver_SP500_stock_prices AS (
    SELECT sp.ticker_symbol
            , c.company_name
            , c.gics_sector
            , c.gics_sub_industry
            , sp.open_price
            , sp.close_price
            , sp.highest_price
            , sp.lowest_price
            , sp.trading_date
    FROM tabular.dataexpert.kdayno_bronze_SP500_stock_prices sp
    INNER JOIN  {catalog_name}.{schema_name}.kdayno_bronze_sp500_companies c ON sp.ticker_symbol = c.ticker_symbol)
    
    SELECT * 
    FROM silver_SP500_stock_prices 
    WHERE trading_date = '{input_load_date}'                        
    """)


In [0]:
spark.sql(f"""
        DELETE FROM {catalog_name}.{schema_name}.kdayno_silver_SP500_stock_prices
        WHERE trading_date = '{input_load_date}'
        """)

(silver_SP500_stock_prices_df.write.format("delta")
                           .mode("append")
                           .partitionBy('trading_date')
                           .saveAsTable(f'{catalog_name}.{schema_name}.kdayno_silver_SP500_stock_prices'))
